# Introduction to tf-shell with AutoParameter Optimization

To get started, `pip install tf-shell`.

`tf-shell` is tightly integrated with TensorFlow's graph execution engine and
extends TensorFlow's graph compiler with some convenient
homomorphic-encryption (HE) specific features.

In this notebook, we will demonstrate how to use `tf-shell` to automatically
choose low level parameters (like the plaintext modulus and ciphertext moduli)
for the BGV HE scheme.
These moduli depend on the depth of a computation, and must be fixed before
the computation starts.
In other examples, these moduli are chosen manually however it is convenient to
let `tf-shell` automatically choose these parameters for you.

In [1]:
import tensorflow as tf
import tf_shell

2024-09-12 15:46:58.912539: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-12 15:46:58.935203: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
a = [1, 2, 3]
b = [4, 5, 6]

Here we define the function we'd like to compute.
TensorFlow will first trace this function (without executing it) to build a
graph of the computation.
Then, during a graph compiler optimization pass, `tf-shell` will replace the
"autoparameters" placeholder with moduli generated for this specific computation
based on statistical estimation of the noise growth, the initial plaintext size,
and the scaling factor.

Note, the `create_autocontext64` function must be called from inside a
`tf.function` in order to execute in non-eager (deferred) mode.
This ensures TensorFlow creates the computation graph which is required to
choose the moduli.

In [3]:
@tf.function
def foo(cleartext_a, cleartext_b):
    shell_context = tf_shell.create_autocontext64(
        log2_cleartext_sz=4,  # Maximum size of the cleartexts (ignoring the scaling factor).
        scaling_factor=1,  # The scaling factor (analagous to fixed-point but not base 2).
        noise_offset_log2=32,  # Extra buffer for noise growth.
    )
    key = tf_shell.create_key64(shell_context)
    a = tf_shell.to_encrypted(cleartext_a, key, shell_context)
    b = tf_shell.to_shell_plaintext(cleartext_b, shell_context)

    intermediate = a * b
    result = tf_shell.to_tensorflow(intermediate, key)
    return result

In [4]:
tf_shell.enable_optimization()

a = [1, 2, 3]
b = [4, 5, 6]
c = foo(a, b)

print(c)

Final parameters:
log_n: 11
t: 12289
qs: 288230376151748609 12289 
tf.Tensor([ 4 10 18 ...  0  0  0], shape=(2048,), dtype=int32)


Recall `tf-shell` treats the first dimension of data as the packing dimension
of the BGV scheme.
In the example above, the three elements of the input vectors are packed into
this first dimension for efficiency purposes.

Note however, that the remaining slots in the ciphertexts went unused and the
number of slots was chosen automatically during graph optimization.